This will use all dataset

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import random

# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

def load_prot_t5_data(pos_file, neg_file):
    """Load ProtT5 embeddings and align with existing data"""
    # Read positive and negative files
    pos_data = []
    with open(pos_file, 'r') as f:
        for line in f:
            parts = line.strip().split(',')
            entry = parts[0]
            pos = int(parts[1])
            embeddings = [float(x) for x in parts[2:]]
            pos_data.append((entry, pos, embeddings))
            
    neg_data = []
    with open(neg_file, 'r') as f:
        for line in f:
            parts = line.strip().split(',')
            entry = parts[0]
            pos = int(parts[1])
            embeddings = [float(x) for x in parts[2:]]
            neg_data.append((entry, pos, embeddings))
    
    # Convert to dictionaries for easy lookup
    pos_dict = {(entry, pos): emb for entry, pos, emb in pos_data}
    neg_dict = {(entry, pos): emb for entry, pos, emb in neg_data}
    
    return pos_dict, neg_dict

def prepare_aligned_data(seq_struct_df, pos_dict, neg_dict):
    """Align ProtT5 embeddings with sequence+structure data"""
    embeddings = []
    aligned_indices = []
    
    for idx, row in seq_struct_df.iterrows():
        key = (row['entry'], row['pos'])
        emb = pos_dict.get(key) if row['label'] == 1 else neg_dict.get(key)
        
        if emb is not None:
            embeddings.append(emb)
            aligned_indices.append(idx)
    
    # Convert to numpy array
    X_prot_t5 = np.array(embeddings)
    
    # Get aligned sequence+structure data
    aligned_df = seq_struct_df.iloc[aligned_indices]
    
    return X_prot_t5, aligned_df

def extract_entry_id(header):
    """Extract entry ID between first and second '|' characters if present, otherwise return as is"""
    if '|' in header:
        try:
            return header.split('|')[1]
        except:
            print(f"Warning: Could not parse header with pipes: {header}")
            return header
    else:
        # If no pipes, assume it's already an ID
        return header

def load_complete_data(mode='train', exclude_emb=False):
    """
    Load data line by line ensuring perfect matching between FASTA and ProtT5 files
    """
    # Set paths based on mode
    if mode == 'train':
        pos_fasta = '../../data/train/fasta/positive_sites.fasta'
        neg_fasta = '../../data/train/fasta/negative_sites.fasta'
        pos_prott5_path = '../../data/train/features/train_positive_ProtT5-XL-UniRef50.csv'
        neg_prott5_path = '../../data/train/features/train_negative_ProtT5-XL-UniRef50.csv'
        struct_path = "../data/processed_features_train_latest.csv"
    else:  # test
        pos_fasta = '../../data/test/fasta/test_positive_sites.fasta'
        neg_fasta = '../../data/test/fasta/test_negative_sites.fasta'
        pos_prott5_path = '../../data/test/features/test_positive_ProtT5-XL-UniRef50.csv'
        neg_prott5_path = '../../data/test/features/test_negative_ProtT5-XL-UniRef50.csv'
        struct_path = "../data/processed_features_test_latest.csv"

    print("Loading structure data first...")
    struct_data = pd.read_csv(struct_path)
    # Clean structure data entry IDs
    struct_data['entry'] = struct_data['entry'].apply(lambda x: extract_entry_id(x) if isinstance(x, str) else x)
    
    # Create dictionary for quick lookup of structure data
    struct_dict = {}
    for _, row in struct_data.iterrows():
        key = (row['entry'], row['pos'])
        struct_dict[key] = row.to_dict()

    print("\nProcessing positive data...")
    positive_data = []
    
    # Process positive data
    with open(pos_fasta) as fasta_file:
        fasta_lines = fasta_file.readlines()
    
    # Read ProtT5 positive data
    prott5_pos = pd.read_csv(pos_prott5_path, header=None)
    
    for i in range(0, len(fasta_lines), 2):
        # Process FASTA header
        header = fasta_lines[i].strip()[1:]  # remove '>'
        sequence = fasta_lines[i + 1].strip()
        
        # Parse header - extract entry ID between | characters and position after |-|
        entry = extract_entry_id(header.split('|-|')[0])
        pos = int(header.split('|-|')[1])
        
        # Get corresponding ProtT5 line
        prott5_line = prott5_pos.iloc[i//2]
        
        # Create data entry
        data_entry = {
            'entry': entry,
            'pos': pos,
            'sequence': sequence,
            'label': 1,
            'has_structure': False
        }
        
        # Add ProtT5 embeddings if requested
        if not exclude_emb:
            for j in range(2, len(prott5_line)):  # skip entry and pos columns
                data_entry[f'emb_{j-2}'] = prott5_line[j]
        
        # Add structure features if available
        if (entry, pos) in struct_dict:
            data_entry['has_structure'] = True
            struct_features = struct_dict[(entry, pos)]
            for key, value in struct_features.items():
                if key not in ['entry', 'pos', 'sequence', 'label']:
                    data_entry[key] = value
        
        positive_data.append(data_entry)
    
    print("Processing negative data...")
    negative_data = []
    
    # Process negative data
    with open(neg_fasta) as fasta_file:
        fasta_lines = fasta_file.readlines()
    
    # Read ProtT5 negative data
    prott5_neg = pd.read_csv(neg_prott5_path, header=None)
    
    for i in range(0, len(fasta_lines), 2):
        # Process FASTA header
        header = fasta_lines[i].strip()[1:]  # remove '>'
        sequence = fasta_lines[i + 1].strip()
        
        # Parse header - extract entry ID between | characters and position after |-|
        entry = extract_entry_id(header.split('|-|')[0])
        pos = int(header.split('|-|')[1])
        
        # Get corresponding ProtT5 line
        prott5_line = prott5_neg.iloc[i//2]
        
        # Create data entry
        data_entry = {
            'entry': entry,
            'pos': pos,
            'sequence': sequence,
            'label': 0,
            'has_structure': False
        }
        
        # Add ProtT5 embeddings if requested
        if not exclude_emb:
            for j in range(2, len(prott5_line)):  # skip entry and pos columns
                data_entry[f'emb_{j-2}'] = prott5_line[j]
        
        # Add structure features if available
        if (entry, pos) in struct_dict:
            data_entry['has_structure'] = True
            struct_features = struct_dict[(entry, pos)]
            for key, value in struct_features.items():
                if key not in ['entry', 'pos', 'sequence', 'label']:
                    data_entry[key] = value
        
        negative_data.append(data_entry)
    
    # Combine all data
    all_data = pd.DataFrame(positive_data + negative_data)
    
    # Check for duplicates
    duplicates = all_data.duplicated(subset=['entry', 'pos'], keep=False)
    if duplicates.any():
        print("\nWarning: Found duplicate entries:")
        print(all_data[duplicates][['entry', 'pos', 'label']].sort_values(['entry', 'pos']))
    
    # Print statistics
    print("\nDataset statistics:")
    print(f"Total entries: {len(all_data)}")
    print(f"Positive examples: {len(positive_data)}")
    print(f"Negative examples: {len(negative_data)}")
    print(f"Entries with structure: {all_data['has_structure'].sum()}")
    print(f"Unique proteins: {all_data['entry'].nunique()}")
    
    # Print some example entries to verify correct parsing
    print("\nExample entries to verify parsing:")
    print(all_data[['entry', 'pos', 'sequence', 'label', 'has_structure']].head())
    
    return all_data

def prepare_sequence_data(df):
    """Convert sequences to integer encoding"""
    alphabet = 'ARNDCQEGHILKMFPSTWYV-'
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    
    sequences = df['sequence'].values
    encodings = []
    
    for seq in sequences:
        try:
            integer_encoded = [char_to_int[char] for char in seq]
            encodings.append(integer_encoded)
        except Exception as e:
            print(f"Error processing sequence: {e}")
            continue
    
    return np.array(encodings)

def prepare_structure_data(df):
    """Enhanced feature preparation focusing on important features"""
    features_list = []
    middle_pos = 16  # Center position
    
    def normalize_angles(angle_array):
        angle_rad = np.pi * angle_array / 180.0
        return np.stack([np.sin(angle_rad), np.cos(angle_rad)], axis=-1)
    
    # Most important features (importance > 0.04)
    important_features = [
        'bfactor',
        'distance_to_center',
        'sasa',
        'omega',
        'domain_position'
    ]
    
    for feature in important_features:
        feature_arrays = np.array([np.array(eval(x)) for x in df[feature]])
        # Take only the center position
        center_values = feature_arrays[:, middle_pos]
        scaler = RobustScaler()
        scaled_values = scaler.fit_transform(center_values.reshape(-1, 1))
        features_list.append(scaled_values)
    
    # Secondary important features (0.03 < importance < 0.04)
    secondary_features = [
        'chi1', 'chi2', 'chi3', 'chi4',
        'curvature', 'psi', 'phi', 'tau',
        'packing_density', 'local_hydrophobicity'
    ]
    
    for feature in secondary_features:
        feature_arrays = np.array([np.array(eval(x)) for x in df[feature]])
        center_values = feature_arrays[:, middle_pos]
        if feature in ['phi', 'psi', 'omega', 'tau']:
            # Angle features get sin/cos encoding
            angle_features = normalize_angles(center_values)
            features_list.append(angle_features)
        else:
            scaler = RobustScaler()
            scaled_values = scaler.fit_transform(center_values.reshape(-1, 1))
            features_list.append(scaled_values)
    
    # Special case: hydrophobicity with ±1 window
    hydro_arrays = np.array([np.array(eval(x)) for x in df['hydrophobicity']])
    hydro_window = hydro_arrays[:, middle_pos-1:middle_pos+2]  # ±1 window
    scaler = RobustScaler()
    hydro_scaled = scaler.fit_transform(hydro_window.reshape(-1, 1)).reshape(len(hydro_arrays), 3)
    features_list.append(hydro_scaled)
    
    # Combine all features
    features = np.concatenate(features_list, axis=-1)
    
    # Add a dimension for the "sequence" length (1 for center position)
    features = features.reshape(features.shape[0], 1, -1)
    
    return features

def create_model_without_structure(seq_length=33):
    """Create model for samples without structure data (sequence + ProtT5 only)"""
    # Sequence track
    seq_input = tf.keras.layers.Input(shape=(seq_length,), name='sequence_input')
    x_seq = tf.keras.layers.Embedding(21, 21, input_length=seq_length)(seq_input)
    x_seq = tf.keras.layers.Reshape((seq_length, 21, 1))(x_seq)
    x_seq = tf.keras.layers.Conv2D(32, kernel_size=(17, 3), activation='relu', padding='valid')(x_seq)
    x_seq = tf.keras.layers.Dropout(0.2)(x_seq)
    x_seq = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x_seq)
    x_seq = tf.keras.layers.Flatten()(x_seq)
    x_seq = tf.keras.layers.Dense(32, activation='relu', name='seq_features')(x_seq)
    x_seq = tf.keras.layers.Dropout(0.2)(x_seq)

    # ProtT5 track (using their architecture)
    prot_t5_input = tf.keras.layers.Input(shape=(1024,), name='prot_t5_input')
    x_prot_t5 = tf.keras.layers.Dense(256)(prot_t5_input)
    x_prot_t5 = tf.keras.layers.Dropout(0.4)(x_prot_t5)
    x_prot_t5 = tf.keras.layers.Dense(128, activation='relu')(x_prot_t5)
    x_prot_t5 = tf.keras.layers.Dropout(0.4)(x_prot_t5)

    # Create learnable weights layer for two tracks
    weight_layer = tf.keras.layers.Dense(2, activation='softmax', name='track_weights')
    track_weights = weight_layer(tf.keras.layers.Concatenate()([x_seq, x_prot_t5]))

    # Apply weights
    weighted_seq = tf.keras.layers.Multiply(name='weighted_seq')([
        x_seq,
        tf.keras.layers.Lambda(lambda x: x[:, 0:1])(track_weights)
    ])
    
    weighted_prot_t5 = tf.keras.layers.Multiply(name='weighted_prot_t5')([
        x_prot_t5,
        tf.keras.layers.Lambda(lambda x: x[:, 1:2])(track_weights)
    ])

    # Combine features
    combined = tf.keras.layers.Concatenate()([weighted_seq, weighted_prot_t5])

    # Final layers
    x = tf.keras.layers.Dense(32, activation='relu')(combined)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[seq_input, prot_t5_input], outputs=outputs)

    def get_track_weights():
        w = model.get_layer('track_weights').get_weights()
        if len(w) > 0:
            weights = w[0]
            bias = w[1] if len(w) > 1 else 0
            exp_weights = np.exp(np.mean(weights, axis=0) + bias)
            normalized = exp_weights / np.sum(exp_weights)
            return {
                'sequence': float(normalized[0]),
                'prot_t5': float(normalized[1])
            }
        return {'sequence': 0.5, 'prot_t5': 0.5}

    model.get_track_weights = get_track_weights
    return model

def create_combined_model_with_prot_t5(seq_length=33, struct_features=None, struct_window=0):
    regularizer = tf.keras.regularizers.l2(0.01)
    
    # Sequence track
    seq_input = tf.keras.layers.Input(shape=(seq_length,), name='sequence_input')
    x_seq = tf.keras.layers.Embedding(21, 21, input_length=seq_length)(seq_input)
    x_seq = tf.keras.layers.Reshape((seq_length, 21, 1))(x_seq)
    x_seq = tf.keras.layers.Conv2D(32, kernel_size=(17, 3), activation='relu', padding='valid')(x_seq)
    x_seq = tf.keras.layers.BatchNormalization()(x_seq)
    x_seq = tf.keras.layers.Dropout(0.4)(x_seq)
    x_seq = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x_seq)
    x_seq = tf.keras.layers.Flatten()(x_seq)
    x_seq = tf.keras.layers.Dense(32, activation='relu', 
                                 kernel_regularizer=regularizer, 
                                 name='seq_features')(x_seq)
    x_seq = tf.keras.layers.BatchNormalization()(x_seq)
    x_seq = tf.keras.layers.Dropout(0.4)(x_seq)

    # Structure track
    struct_input = tf.keras.layers.Input(shape=(1, struct_features), name='structure_input')
    x_struct = struct_input
    struct_dense_size = min(struct_features * 2, 128)
    
    x_struct = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x_struct)
    x_struct = tf.keras.layers.BatchNormalization()(x_struct)
    x_struct = tf.keras.layers.Dropout(0.4)(x_struct)
    x_struct = tf.keras.layers.Flatten()(x_struct)
    x_struct = tf.keras.layers.Dense(struct_dense_size, activation='relu',
                                   kernel_regularizer=regularizer)(x_struct)
    x_struct = tf.keras.layers.BatchNormalization()(x_struct)
    x_struct = tf.keras.layers.Dropout(0.4)(x_struct)

    # ProtT5 track
    prot_t5_input = tf.keras.layers.Input(shape=(1024,), name='prot_t5_input')
    x_prot_t5 = tf.keras.layers.Dense(256, kernel_regularizer=regularizer)(prot_t5_input)
    x_prot_t5 = tf.keras.layers.BatchNormalization()(x_prot_t5)
    x_prot_t5 = tf.keras.layers.Dropout(0.5)(x_prot_t5)
    x_prot_t5 = tf.keras.layers.Dense(128, activation='relu',
                                     kernel_regularizer=regularizer)(x_prot_t5)
    x_prot_t5 = tf.keras.layers.BatchNormalization()(x_prot_t5)
    x_prot_t5 = tf.keras.layers.Dropout(0.5)(x_prot_t5)

    # Create learnable weights layer
    weight_layer = tf.keras.layers.Dense(3, activation='softmax', name='track_weights')
    track_weights = weight_layer(tf.keras.layers.Concatenate()([x_seq, x_struct, x_prot_t5]))

    # Apply weights
    weighted_seq = tf.keras.layers.Multiply(name='weighted_seq')([
        x_seq,
        tf.keras.layers.Lambda(lambda x: x[:, 0:1])(track_weights)
    ])
    
    weighted_struct = tf.keras.layers.Multiply(name='weighted_struct')([
        x_struct,
        tf.keras.layers.Lambda(lambda x: x[:, 1:2])(track_weights)
    ])
    
    weighted_prot_t5 = tf.keras.layers.Multiply(name='weighted_prot_t5')([
        x_prot_t5,
        tf.keras.layers.Lambda(lambda x: x[:, 2:3])(track_weights)
    ])

    # Combine features
    combined = tf.keras.layers.Concatenate()([weighted_seq, weighted_struct, weighted_prot_t5])

    # Final layers with more regularization
    x = tf.keras.layers.Dense(64, activation='relu', 
                            kernel_regularizer=regularizer)(combined)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(32, activation='relu', 
                            kernel_regularizer=regularizer)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[seq_input, struct_input, prot_t5_input], outputs=outputs)
    
    def get_track_weights():
        w = model.get_layer('track_weights').get_weights()
        if len(w) > 0:
            weights = w[0]
            bias = w[1] if len(w) > 1 else 0
            exp_weights = np.exp(np.mean(weights, axis=0) + bias)
            normalized = exp_weights / np.sum(exp_weights)
            return {
                'sequence': float(normalized[0]),
                'structure': float(normalized[1]),
                'prot_t5': float(normalized[2])
            }
        return {'sequence': 0.33, 'structure': 0.33, 'prot_t5': 0.34}

    model.get_track_weights = get_track_weights
    return model

def print_final_results(track_weights_history, metrics, test_predictions, y_test):
    """Print final results and statistics"""
    # Calculate average weights
    avg_seq_weight = np.mean([w['sequence'] for w in track_weights_history])
    avg_struct_weight = np.mean([w['structure'] for w in track_weights_history])
    avg_prot_t5_weight = np.mean([w['prot_t5'] for w in track_weights_history])
    std_seq_weight = np.std([w['sequence'] for w in track_weights_history])
    std_struct_weight = np.std([w['structure'] for w in track_weights_history])
    std_prot_t5_weight = np.std([w['prot_t5'] for w in track_weights_history])
    
    print("\nAverage track weights across folds:")
    print(f"Sequence weight: {avg_seq_weight:.4f} ± {std_seq_weight:.4f}")
    print(f"Structure weight: {avg_struct_weight:.4f} ± {std_struct_weight:.4f}")
    print(f"ProtT5 weight: {avg_prot_t5_weight:.4f} ± {std_prot_t5_weight:.4f}")
    
    # Print average cross-validation results
    print("\nAverage Cross-validation Results:")
    for metric in metrics:
        print(f"{metric.upper()}: {np.mean(metrics[metric]):.4f} ± {np.std(metrics[metric]):.4f}")
    
    # Calculate final test results
    test_pred_avg = np.mean(test_predictions, axis=0)
    test_pred_binary = (test_pred_avg > 0.5).astype(int)
    cm_test = confusion_matrix(y_test, test_pred_binary)
    
    print("\nFinal Test Set Results:")
    print(f"Accuracy: {accuracy_score(y_test, test_pred_binary):.4f}")
    print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, test_pred_binary):.4f}")
    print(f"MCC: {matthews_corrcoef(y_test, test_pred_binary):.4f}")
    print(f"Sensitivity: {cm_test[1][1]/(cm_test[1][1]+cm_test[1][0]):.4f}")
    print(f"Specificity: {cm_test[0][0]/(cm_test[0][0]+cm_test[0][1]):.4f}")
    print("Confusion Matrix:")
    print(cm_test)
    
def train_and_evaluate_split():
    """Training function that handles both cases with k-fold validation"""
    # Load complete data without embeddings
    print("Loading complete data...")
    train_data = load_complete_data(mode='train', exclude_emb=True)
    test_data = load_complete_data(mode='test', exclude_emb=True)
    
    # Load ProtT5 embeddings
    print("Loading ProtT5 embeddings...")
    train_pos_dict, train_neg_dict = load_prot_t5_data(
        '/Users/hai/Workspace/UniFreiburg/WS2024/Thesis/LMSuccSite/data/train/features/train_positive_ProtT5-XL-UniRef50.csv',
        '/Users/hai/Workspace/UniFreiburg/WS2024/Thesis/LMSuccSite/data/train/features/train_negative_ProtT5-XL-UniRef50.csv'
    )
    test_pos_dict, test_neg_dict = load_prot_t5_data(
        '/Users/hai/Workspace/UniFreiburg/WS2024/Thesis/LMSuccSite/data/test/features/test_positive_ProtT5-XL-UniRef50.csv',
        '/Users/hai/Workspace/UniFreiburg/WS2024/Thesis/LMSuccSite/data/test/features/test_negative_ProtT5-XL-UniRef50.csv'
    )
    
    # Align data with ProtT5 embeddings
    print("Aligning data with ProtT5 embeddings...")
    X_train_prot_t5, train_data_aligned = prepare_aligned_data(train_data, train_pos_dict, train_neg_dict)
    X_test_prot_t5, test_data_aligned = prepare_aligned_data(test_data, test_pos_dict, test_neg_dict)
    
    # Split data by structure availability
    train_with_struct = train_data_aligned[train_data_aligned['has_structure']].copy()
    train_without_struct = train_data_aligned[~train_data_aligned['has_structure']].copy()
    test_with_struct = test_data_aligned[test_data_aligned['has_structure']].copy()
    test_without_struct = test_data_aligned[~test_data_aligned['has_structure']].copy()
    
    print("\nData Statistics:")
    print(f"Total training samples: {len(train_data_aligned)}")
    print(f"Training samples with structure: {len(train_with_struct)} ({len(train_with_struct)/len(train_data_aligned)*100:.1f}%)")
    print(f"Total test samples: {len(test_data_aligned)}")
    print(f"Test samples with structure: {len(test_with_struct)} ({len(test_with_struct)/len(test_data_aligned)*100:.1f}%)")
    
    # Prepare sequence data for all samples
    X_train_seq_all = prepare_sequence_data(train_data_aligned)
    X_test_seq_all = prepare_sequence_data(test_data_aligned)
    y_train_all = train_data_aligned['label'].values
    y_test_all = test_data_aligned['label'].values
    
    # Prepare data for samples with structure
    if len(train_with_struct) > 0:
        X_train_seq_struct = prepare_sequence_data(train_with_struct)
        X_train_struct = prepare_structure_data(train_with_struct)
        X_train_prot_t5_struct = X_train_prot_t5[train_data_aligned['has_structure']]
        y_train_struct = train_with_struct['label'].values
        
        X_test_seq_struct = prepare_sequence_data(test_with_struct)
        X_test_struct = prepare_structure_data(test_with_struct)
        X_test_prot_t5_struct = X_test_prot_t5[test_data_aligned['has_structure']]
        y_test_struct = test_with_struct['label'].values
    
    # Prepare data for samples without structure
    X_train_seq_no_struct = prepare_sequence_data(train_without_struct)
    X_train_prot_t5_no_struct = X_train_prot_t5[~train_data_aligned['has_structure']]
    y_train_no_struct = train_without_struct['label'].values
    
    X_test_seq_no_struct = prepare_sequence_data(test_without_struct)
    X_test_prot_t5_no_struct = X_test_prot_t5[~test_data_aligned['has_structure']]
    y_test_no_struct = test_without_struct['label'].values
    
    # Calculate class weights
    total_samples = len(y_train_all)
    pos_samples = np.sum(y_train_all == 1)
    neg_samples = np.sum(y_train_all == 0)
    class_weights = {
        0: total_samples / (2 * neg_samples),
        1: total_samples / (2 * pos_samples)
    }
    
    # Print class distribution
    print("\nTraining set distribution:")
    print(pd.Series(y_train_all).value_counts())
    print("\nTest set distribution:")
    print(pd.Series(y_test_all).value_counts())
    
    # Print data shapes
    print("\nData shapes:")
    print(f"X_train_seq_all: {X_train_seq_all.shape}")
    print(f"X_train_prot_t5: {X_train_prot_t5.shape}")
    if len(train_with_struct) > 0:
        print(f"X_train_struct: {X_train_struct.shape}")
    
    # Initialize k-fold
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    metrics = {'acc': [], 'balanced_acc': [], 'mcc': [], 'sn': [], 'sp': []}
    test_predictions = []
    track_weights_history = []
    
    # Training callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-6
        )
    ]
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_seq_all), 1):
        print(f"\nFold {fold}/5")
        
        # Train sequence+ProtT5 model on all data
        print("\nTraining sequence+ProtT5 model...")
        model_no_struct = create_model_without_structure(seq_length=33)
        model_no_struct.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        history_no_struct = model_no_struct.fit(
            [X_train_seq_all[train_idx], X_train_prot_t5[train_idx]],
            y_train_all[train_idx],
            validation_data=(
                [X_train_seq_all[val_idx], X_train_prot_t5[val_idx]],
                y_train_all[val_idx]
            ),
            batch_size=32,
            epochs=50,
            callbacks=callbacks,
            class_weight=class_weights,
            verbose=1
        )
        
        # Train full model with structure
        if len(train_with_struct) > 0:
            print("\nTraining full model...")
            # Find indices for structure data
            struct_mask = train_data_aligned['has_structure'].values
            struct_train_idx = train_idx[struct_mask[train_idx]]
            struct_val_idx = val_idx[struct_mask[val_idx]]
            
            model_with_struct = create_combined_model_with_prot_t5(
                seq_length=33,
                struct_features=X_train_struct.shape[2],
                struct_window=0
            )
            
            model_with_struct.compile(
                optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )
            
            history_struct = model_with_struct.fit(
                [X_train_seq_struct[struct_train_idx], X_train_struct[struct_train_idx], X_train_prot_t5_struct[struct_train_idx]],
                y_train_struct[struct_train_idx],
                validation_data=(
                    [X_train_seq_struct[struct_val_idx], X_train_struct[struct_val_idx], X_train_prot_t5_struct[struct_val_idx]],
                    y_train_struct[struct_val_idx]
                ),
                batch_size=32,
                epochs=50,
                callbacks=callbacks,
                class_weight=class_weights,
                verbose=1
            )
            
            # Store track weights
            track_weights = model_with_struct.get_track_weights()
            track_weights_history.append(track_weights)
            print(f"\nLearned track weights for fold {fold}:")
            print(f"Sequence weight: {track_weights['sequence']:.4f}")
            print(f"Structure weight: {track_weights['structure']:.4f}")
            print(f"ProtT5 weight: {track_weights['prot_t5']:.4f}")
        
        # Plot training histories
        plt.figure(figsize=(15, 5))
        
        # Plot sequence+ProtT5 model history
        plt.subplot(1, 2, 1)
        plt.plot(history_no_struct.history['accuracy'], label='Train')
        plt.plot(history_no_struct.history['val_accuracy'], label='Val')
        plt.title(f'Seq+ProtT5 Model - Fold {fold}')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        # Plot full model history if available
        if len(train_with_struct) > 0:
            plt.subplot(1, 2, 2)
            plt.plot(history_struct.history['accuracy'], label='Train')
            plt.plot(history_struct.history['val_accuracy'], label='Val')
            plt.title(f'Full Model - Fold {fold}')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.legend()
        
        plt.tight_layout()
        plt.show()
        
        # Get predictions for test set
        fold_predictions = []
        fold_labels = []
        
        # Predictions for samples with structure
        if len(test_with_struct) > 0:
            pred_struct = model_with_struct.predict(
                [X_test_seq_struct, X_test_struct, X_test_prot_t5_struct]
            )
            fold_predictions.extend(pred_struct)
            fold_labels.extend(y_test_struct)
        
        # Predictions for samples without structure
        if len(test_without_struct) > 0:
            pred_no_struct = model_no_struct.predict(
                [X_test_seq_no_struct, X_test_prot_t5_no_struct]
            )
            fold_predictions.extend(pred_no_struct)
            fold_labels.extend(y_test_no_struct)
        
        # Convert to numpy arrays
        fold_predictions = np.array(fold_predictions)
        fold_labels = np.array(fold_labels)
        
        # Calculate metrics for this fold
        fold_pred_binary = (fold_predictions > 0.5).astype(int)
        cm = confusion_matrix(fold_labels, fold_pred_binary)
        
        metrics['acc'].append(accuracy_score(fold_labels, fold_pred_binary))
        metrics['balanced_acc'].append(balanced_accuracy_score(fold_labels, fold_pred_binary))
        metrics['mcc'].append(matthews_corrcoef(fold_labels, fold_pred_binary))
        metrics['sn'].append(cm[1][1]/(cm[1][1]+cm[1][0]))  # Sensitivity
        metrics['sp'].append(cm[0][0]/(cm[0][0]+cm[0][1]))  # Specificity
        
        # Store predictions
        test_predictions.append(fold_predictions)
        
        # Print fold results
        print(f"\nFold {fold} Results:")
        print(f"Accuracy: {metrics['acc'][-1]:.4f}")
        print(f"Balanced Accuracy: {metrics['balanced_acc'][-1]:.4f}")
        print(f"MCC: {metrics['mcc'][-1]:.4f}")
        print(f"Sensitivity: {metrics['sn'][-1]:.4f}")
        print(f"Specificity: {metrics['sp'][-1]:.4f}")
    
    # Calculate and print final results
    test_pred_avg = np.mean(test_predictions, axis=0)
    test_pred_binary = (test_pred_avg > 0.5).astype(int)
    cm_test = confusion_matrix(y_test_all, test_pred_binary)
    
    print("\nAverage Cross-validation Results:")
    for metric in metrics:
        print(f"{metric.upper()}: {np.mean(metrics[metric]):.4f} ± {np.std(metrics[metric]):.4f}")
    
    print("\nFinal Test Set Results:")
    print(f"Accuracy: {accuracy_score(y_test_all, test_pred_binary):.4f}")
    print(f"Balanced Accuracy: {balanced_accuracy_score(y_test_all, test_pred_binary):.4f}")
    print(f"MCC: {matthews_corrcoef(y_test_all, test_pred_binary):.4f}")
    print(f"Sensitivity: {cm_test[1][1]/(cm_test[1][1]+cm_test[1][0]):.4f}")
    print(f"Specificity: {cm_test[0][0]/(cm_test[0][0]+cm_test[0][1]):.4f}")
    print("\nConfusion Matrix:")
    print(cm_test)
    
    if track_weights_history:
        # Print average track weights
        print("\nAverage track weights across folds:")
        avg_weights = {
            'sequence': np.mean([w['sequence'] for w in track_weights_history]),
            'structure': np.mean([w['structure'] for w in track_weights_history]),
            'prot_t5': np.mean([w['prot_t5'] for w in track_weights_history])
        }
        std_weights = {
            'sequence': np.std([w['sequence'] for w in track_weights_history]),
            'structure': np.std([w['structure'] for w in track_weights_history]),
            'prot_t5': np.std([w['prot_t5'] for w in track_weights_history])
        }
        
        print(f"Sequence weight: {avg_weights['sequence']:.4f} ± {std_weights['sequence']:.4f}")
        print(f"Structure weight: {avg_weights['structure']:.4f} ± {std_weights['structure']:.4f}")
        print(f"ProtT5 weight: {avg_weights['prot_t5']:.4f} ± {std_weights['prot_t5']:.4f}")
    
    return model_with_struct, model_no_struct

if __name__ == "__main__":
    model_with_struct, model_no_struct = train_and_evaluate_split()

Loading complete data...
Loading structure data first...

Processing positive data...
Processing negative data...

       entry   pos  label
5149  A2A274   574      0
9481  A2A274   574      0
6617  A2A274   725      0
8100  A2A274   725      0
5098  A2A7A7   331      0
...      ...   ...    ...
8637  V4Z729   112      0
4957  V4Z7M7   122      0
6616  V4Z7M7   122      0
7535  V4Z7M7  2271      0
7586  V4Z7M7  2271      0

[435 rows x 3 columns]

Dataset statistics:
Total entries: 9500
Positive examples: 4750
Negative examples: 4750
Entries with structure: 8853
Unique proteins: 2193

Example entries to verify parsing:
    entry  pos                           sequence  label  has_structure
0  P0A823  227  EILAYKAEISAEGMALKKSLPVTL---------      1           True
1  P75863  344  CGSCRVQLLEGEVTPLKKSAMGDDGTILCCSCV      1           True
2  P0ABS8    7  ----------MLKNLAKLDQTEMDKVNVDLAAA      1           True
3  P0ABS8   74  AHRLASVNLSRLPYEPKLK--------------      1           True
4  P0A7D7    

/Users/hai/miniconda3/envs/bioinf/lib/python3.9/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2024-12-12 05:14:01.925439: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-12 05:14:01.925470: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-12 05:14:01.925479: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-12 05:14:01.925493: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-12 05:14:01.925504: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/50


2024-12-12 05:14:02.646776: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


238/238 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.6077 - loss: 0.6520 - val_accuracy: 0.6989 - val_loss: 0.5692 - learning_rate: 0.0010
Epoch 2/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7215 - loss: 0.5599 - val_accuracy: 0.7142 - val_loss: 0.5586 - learning_rate: 0.0010
Epoch 3/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7437 - loss: 0.5311 - val_accuracy: 0.7195 - val_loss: 0.5589 - learning_rate: 0.0010
Epoch 4/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - accuracy: 0.7500 - loss: 0.5037 - val_accuracy: 0.7232 - val_loss: 0.5485 - learning_rate: 0.0010
Epoch 5/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7661 - loss: 0.4804 - val_accuracy: 0.7342 - val_loss: 0.5469 - learning_rate: 0.0010
Epoch 6/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7837 - loss: 0.4646 - val_accuracy: 0.7547 - val_loss: 0.5375 - learning_rate: 0.0010
Epoch 7/50
238/238 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.8019 - loss: 0.4337 - val_

IndexError: index 8853 is out of bounds for axis 0 with size 8853